# artsy

In [1]:
import json

from pymongo import MongoClient
import requests

import credentials

## setup

### database

In [23]:
client = MongoClient()
client.drop_database('artsy')
db = client.artsy

### base url

In [3]:
BASE_URL = 'https://api.artsy.net/api'

### app

In [4]:
app_name = credentials.artsy_app_name
client_id = credentials.artsy_client_id
client_secret = credentials.artsy_client_secret

### token

In [5]:
headers = {}
payload = {'client_id': client_id, 'client_secret': client_secret}
url = BASE_URL + '/tokens/xapp_token'
response = requests.post(url, params=payload, headers=headers)
token = response.json()['token']

### auxiliar

In [43]:
def change_id_field(d):
    d['_id'] = d.pop('id')

### test

In [44]:
# headers = {'X-Xapp-Token': token}x
# payload = {'total_count': 1}
# url = BASE_URL + '/'
# response = requests.get(url, params=payload, headers=headers)
# # total_count = response.json()['total_count']
# print(response.json())

SyntaxError: invalid syntax (<ipython-input-44-11a72e4efb48>, line 1)

## artsy resources
* Status
* Artists
* Artworks
* Editions
* Genes
* Profiles
* Partners
* Shows
* Fairs
* Applications
* Users
* User Details
* Collections
* Collection Items
* Images
* Search

## status

In [48]:
headers = {}
payload = {'total_count': 1}
url = BASE_URL + '/status'
response = requests.get(url, params=payload, headers=headers)
status = response.json()['status']
print('Status: %s' % status)

Status: good


## resources
artists, artworks, genes, profiles, partners, shows, fairs

In [73]:
resources = ['artists', 'artworks', 'genes', 'profiles', 'partners', 'shows', 'fairs']
for resource in resources:
    print('Processing %s...', resource)
    url = BASE_URL + '/' + resource
    headers = {'X-Xapp-Token': token}
    
    # total count
    payload = {'total_count': 1}
    response = requests.get(url, params=payload, headers=headers)
    total_count = response.json()['total_count']
    print('Total %s: %d' % (resource, total_count))
    
    # data
    payload = {}
    count = 0
    while True:
        response = requests.get(url, params=payload, headers=headers)
        data = response.json()['_embedded'][resource]
        if not data:
            break
        [change_id_field(d) for d in data]
        db.artists.insert_many(data)
        url = response.json()['_links']['next']['href']
        if not url:
            break
        count += len(data)
        if count % (total_count / 10) < len(data):
            print('%d %s processed.' % (resource, count))

KeyError: 'total_count'

## search

In [31]:
search_term = 'Andy Warhol'

In [32]:
search_url = BASE_URL + '/search'
headers = {'X-Xapp-Token': token}
payload = {'q': search_term}
response = requests.get(search_url, params=payload, headers=headers)

In [33]:
response

<Response [200]>

In [34]:
response.json()

{'_embedded': {'results': [{'_links': {'permalink': {'href': 'https://www.artsy.net/artist/andy-warhol'},
     'self': {'href': 'https://api.artsy.net/api/artists/andy-warhol'},
     'thumbnail': {'href': 'https://d32dm0rphc51dk.cloudfront.net/PFufT6nMKNwLOpPEezf4Ww/large.jpg'}},
    'description': 'Dec 5, 2015 ... Browse the best of Andy Warhol art, including paintings and artwork for sale (with \nprices), upcoming shows, and exclusive Andy Warhol articles\xa0...',
    'og_type': 'artsyinc:artist',
    'title': 'Andy Warhol Art - 1161 Artworks, Biography & Shows on Artsy',
    'type': 'Artist'},
   {'_links': {'permalink': {'href': 'https://www.artsy.net/artist/andy-warhol/works'},
     'self': {'href': 'https://api.artsy.net/api/artists/andy-warhol'},
     'thumbnail': {'href': 'https://i.embed.ly/1/display/resize?height=300&quality=95&grow=false&url=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FVFelegc1J0GFxkS2LSJdRg%2Flarger.jpg&key=a1f82558d8134f6cbebceb9e67d04980'}},
    'descrip